In [1]:
!pip -qqq install pip --progress-bar off
!pip -qqq install langchain-groq==0.1.3 --progress-bar off
!pip -qqq install langchain==0.1.17 --progress-bar off
!pip -qqq install llama-parse==0.1.3 --progress-bar off
!pip -qqq install qdrant-client==1.9.1  --progress-bar off
!pip -qqq install "unstructured[md]"==0.13.6 --progress-bar off
!pip -qqq install fastembed==0.2.7 --progress-bar off
!pip -qqq install flashrank==0.2.4 --progress-bar off


  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.41.2 requires huggingface-hub<1.0,>=0.23.0, but you have huggingface-hub 0.20.3 which is incompatible.
transformers 4.41.2 requires tokenizers<0.20,>=0.19, but you have tokenizers 0.15.2 which is incompatible.


In [2]:
import os
import textwrap
from google.colab import userdata
from IPython.display import Markdown
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from llama_parse import LlamaParse


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install groq


In [5]:
os.environ["GROQ_API_KEY"] = ""

def print_response(response):
    response_txt = response["result"]
    for chunk in response_txt.split("\n"):
        if not chunk:
            print()
            continue
        print("\n".join(textwrap.wrap(chunk, 100, break_long_words=False)))



# Document parsing

In [7]:
#Document parsing (cleaning the document)
instruction = """The provided document is the Constitution of Nepal 2072. This document encompasses all the legal frameworks, guidelines, and principles governing the country. It defines the structure of the state, the distribution of powers between different levels of government, and the fundamental rights and duties of citizens.

When answering questions based on this document, please follow these guidelines:
- Be precise and concise in your responses.
- Ensure that the information is accurate and directly relevant to the question.
- Highlight key articles, sections, or provisions where applicable.
- Provide context to your answers when necessary, explaining the implications or importance of specific provisions.
- Maintain a neutral and informative tone, avoiding any personal opinions or interpretations.

The goal is to provide clear and informative answers that help the user understand the specific aspects of the Constitution of Nepal 2072."""

parser = LlamaParse(
    api_key = '',
    result_type = 'markdown',
    parsing_instruction = instruction,
    max_timeout = 5000,

)

llama_parse_document = await parser.aload_data('/content/drive/My Drive/Constitution_Nepal.pdf')

Started parsing the file under job_id 17389fca-066d-423a-9410-c38058eccf1b


In [8]:
parsed_doc = llama_parse_document[0]
Markdown(parsed_doc.text[:1000])

# The Constitution of Nepal

Date of Publication in Nepal Gazette

20 September 2015 (2072.6.3)

The Constitution of Nepal (First Amendment), 2016 (2072)

28 February 2016 (2072.11.16)

Preamble:

We, the Sovereign People of Nepal, Internalizing the people's sovereign right and right to autonomy and self-rule, while maintaining freedom, sovereignty, territorial integrity, national unity, independence and dignity of Nepal, Recalling the glorious history of historic people's movements, armed conflict, dedication and sacrifice undertaken by the Nepalese people at times for the interest of the nation, democracy and progressive changes, and respecting for the martyrs and disappeared and victim citizens, Ending all forms of discrimination and oppression created by the feudalistic, autocratic, centralized, unitary system of governance, Protecting and promoting social and cultural solidarity, tolerance and harmony, and unity in diversity by recognizing the multi-ethnic, multi-lingual, multi-re

In [9]:
from pathlib import Path

In [11]:
# save the file to
document_path = Path('/content/drive/MyDrive/parsed_document.md')
with document_path.open("a") as f:
  f.write(parsed_doc.text)

# Vector embedding

In [12]:
# Handle markdown file
loader = UnstructuredMarkdownLoader(document_path)
loaded_documents = loader.load()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [13]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=128)
docs = text_splitter.split_documents(loaded_documents)
print(docs[0].page_content)

The Constitution of Nepal

Date of Publication in Nepal Gazette 20 September 2015 (2072.6.3) The Constitution of Nepal (First Amendment), 2016 (2072) 28 February 2016 (2072.11.16)

Preamble:

We, the Sovereign People of Nepal, Internalizing the people's sovereign right and right to autonomy and self-rule, while maintaining freedom, sovereignty, territorial integrity, national unity, independence and dignity of Nepal, Recalling the glorious history of historic people's movements, armed conflict, dedication and sacrifice undertaken by the Nepalese people at times for the interest of the nation, democracy and progressive changes, and respecting for the martyrs and disappeared and victim citizens, Ending all forms of discrimination and oppression created by the feudalistic, autocratic, centralized, unitary system of governance, Protecting and promoting social and cultural solidarity, tolerance and harmony, and unity in diversity by recognizing the multi-ethnic, multi-lingual, multi-religio

In [14]:
# model embedding
embeddings = FastEmbedEmbeddings(
    model_name = "BAAI/bge-base-en-v1.5")
qdrant = Qdrant.from_documents(docs,
                               embeddings,
                               path = '/content/drive/MyDrive/qdrant_db',
                               collection_name = "document_embeddings")


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/740 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/218M [00:00<?, ?B/s]

In [26]:
%%time
query = "What fundamental rights are guaranteed to Nepali citizens?"
similar_docs = qdrant.similarity_search_with_score(query)

CPU times: user 321 ms, sys: 5 µs, total: 321 ms
Wall time: 318 ms


In [27]:
for doc, score in similar_docs:
    print(f"text: {doc.page_content[:256]}\n")
    print(f"score: {score}")
    print("_" * 80)
    print()

text: Right to privacy

The privacy of any person, his or her residence, property, document, data, correspondence and matters relating to his or her character shall, except in accordance with law, be inviolable.

Right against exploitation

Every person shall ha

score: 0.780478909497755
________________________________________________________________________________

text: Every person shall have the right to a fair trial by an independent, impartial and competent court or judicial body.

Any indigent party shall have the right to free legal aid in accordance with law.

Right of victim of crime:

(1) A victim of crime shall 

score: 0.779577503923492
________________________________________________________________________________

text: Any indigent party shall have the right to free legal aid in accordance with law.

Right of victim of crime:

(1) A victim of crime shall have pe right to get information about pe investigation and proceedings of a case in which he or she is pe victim.

In [28]:
%%time
retriever = qdrant.as_retriever(search_kwargs = {"k": 5})
retrieved_docs = retriever.invoke(query)

CPU times: user 343 ms, sys: 0 ns, total: 343 ms
Wall time: 346 ms


In [29]:
for doc in retrieved_docs:
  print(f"id: {doc.metadata['_id']}\n")
  print(f"text: {doc.page_content[:256]}")
  print("_" * 80)
  print()

id: b25ab55d3c00447a96a11f805c0cf23d

text: Right to privacy

The privacy of any person, his or her residence, property, document, data, correspondence and matters relating to his or her character shall, except in accordance with law, be inviolable.

Right against exploitation

Every person shall ha
________________________________________________________________________________

id: 9db2e8e498fd4ed1a6607fe9b81f434f

text: Every person shall have the right to a fair trial by an independent, impartial and competent court or judicial body.

Any indigent party shall have the right to free legal aid in accordance with law.

Right of victim of crime:

(1) A victim of crime shall 
________________________________________________________________________________

id: 0b2887a1eae843709ea42121a7cee6d8

text: Any indigent party shall have the right to free legal aid in accordance with law.

Right of victim of crime:

(1) A victim of crime shall have pe right to get information about pe investigat

In [21]:
# path = '/content/drive/MyDrive/LLM-Nepal-constitution'

### Reranking

In [30]:
# Rerank and compress
compressor = FlashrankRerank(model = 'ms-marco-MiniLM-L-12-v2')
compression_retriever = ContextualCompressionRetriever(
    base_compressor = compressor,
    base_retriever = retriever)

In [31]:
%%time
reranked_docs = compression_retriever.invoke(query)
len(reranked_docs)

Running pairwise ranking..
CPU times: user 2.12 s, sys: 288 ms, total: 2.41 s
Wall time: 2.28 s


3

In [32]:
for doc in reranked_docs:
    print(f"id: {doc.metadata['_id']}\n")
    print(f"text: {doc.page_content[:256]}\n")
    print(f"score: {doc.metadata['relevance_score']}")
    print("-" * 80)
    print()

id: b25ab55d3c00447a96a11f805c0cf23d

text: Right to privacy

The privacy of any person, his or her residence, property, document, data, correspondence and matters relating to his or her character shall, except in accordance with law, be inviolable.

Right against exploitation

Every person shall ha

score: 0.9319560527801514
--------------------------------------------------------------------------------

id: a53d2f0d5ccf4e9fbe8c43873b8398d1

text: Right to social security: The indigent citizens, incapacitated and helpless citizens, helpless single women, citizens with disabilities, children, citizens who cannot take care themselves and citizens belonging to the tribes on the verge of extinction shal

score: 0.7536115050315857
--------------------------------------------------------------------------------

id: 0b2887a1eae843709ea42121a7cee6d8

text: Any indigent party shall have the right to free legal aid in accordance with law.

Right of victim of crime:

(1) A victim of crime shal

In [33]:
llm = ChatGroq(temperature = 0, model = 'llama3-70b-8192')

In [34]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Answer the question and provide additional helpful information,
based on the pieces of information, if applicable. Be succinct.

Responses should be properly formatted to be easily read.
"""

prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [35]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt, "verbose" : True},
)

In [36]:
%%time

response = qa.invoke("what are the fundamental rights guaranteed to Nepali citizens?")

Running pairwise ranking..


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: 15. Other provisions relating to citizenship of Nepal: Other matters relating to the maintenance of records setting out the identity of every citizen of Nepal and the citizenship of Nepal shall be as provided for in the Federal law.

Part-3 Fundamental Rights and Duties

Right to live with dignity:

Every person shall have the right to live with dignity.

No law shall be made providing for the death penalty to any one.

Right to freedom:

No person shall be deprived of his or her personal liberty except in accordance with law.

Every citizen shall have the following freedoms:

(a) freedom of opinion and expression,

(b) freedom to assemble peaceably and without arms,

(c) freedom to f

In [37]:
print_response(response)

**Fundamental Rights Guaranteed to Nepali Citizens:**

1. **Right to Live with Dignity**: Every person has the right to live with dignity, and no law shall
provide for the death penalty.
2. **Right to Freedom**: No person shall be deprived of personal liberty except in accordance with
law, and citizens have the following freedoms:
        * Freedom of opinion and expression
        * Freedom to assemble peaceably and without arms
        * Freedom to form political parties
        * Freedom to form unions and associations
        * Freedom to move and reside in any part of Nepal
        * Freedom to practice any profession, carry on any occupation, and establish and operate any
industry, trade, and business in any part of Nepal
3. **Right to Privacy**: The privacy of any person, their residence, property, document, data,
correspondence, and matters relating to their character shall be inviolable, except in accordance
with law.
4. **Right Against Exploitation**: Every person has the rig

In [38]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt, "verbose" : False},
)

In [39]:
%%time

response = qa.invoke("what are the fundamental rights guaranteed to Nepali citizens?")

Running pairwise ranking..
CPU times: user 2.43 s, sys: 161 ms, total: 2.59 s
Wall time: 5.29 s


In [40]:
print_response(response)

**Fundamental Rights Guaranteed to Nepali Citizens:**

1. **Right to Live with Dignity**: Every person has the right to live with dignity, and no law shall
provide for the death penalty.
2. **Right to Freedom**: No person shall be deprived of personal liberty except in accordance with
law, and citizens have the following freedoms:
        * Freedom of opinion and expression
        * Freedom to assemble peaceably and without arms
        * Freedom to form political parties
        * Freedom to form unions and associations
        * Freedom to move and reside in any part of Nepal
        * Freedom to practice any profession, carry on any occupation, and establish and operate any
industry, trade, and business in any part of Nepal
3. **Right to Privacy**: The privacy of any person, their residence, property, document, data,
correspondence, and matters relating to their character shall be inviolable, except in accordance
with law.
4. **Right Against Exploitation**: Every person has the rig

In [41]:
Markdown(response["result"])

**Fundamental Rights Guaranteed to Nepali Citizens:**

1. **Right to Live with Dignity**: Every person has the right to live with dignity, and no law shall provide for the death penalty.
2. **Right to Freedom**: No person shall be deprived of personal liberty except in accordance with law, and citizens have the following freedoms:
	* Freedom of opinion and expression
	* Freedom to assemble peaceably and without arms
	* Freedom to form political parties
	* Freedom to form unions and associations
	* Freedom to move and reside in any part of Nepal
	* Freedom to practice any profession, carry on any occupation, and establish and operate any industry, trade, and business in any part of Nepal
3. **Right to Privacy**: The privacy of any person, their residence, property, document, data, correspondence, and matters relating to their character shall be inviolable, except in accordance with law.
4. **Right Against Exploitation**: Every person has the right against exploitation, and no one shall be subjected to trafficking, slavery, or servitude, or forced to work against their will.
5. **Right to Clean Environment**: Every citizen has the right to live in a clean and healthy environment and to obtain compensation for any injury caused by environmental pollution or degradation.
6. **Right Relating to Education**: Every citizen has the right to access basic education, compulsory and free education up to the basic level, and free education up to the secondary level from the State.
7. **Right to Language and Culture**: Every Nepalese community residing in Nepal has the right to get education in its mother tongue and to open and operate schools and educational institutes.
8. **Right of Victim of Crime**: A victim of crime has the right to get information about the investigation and proceedings of a case, and to get justice, including social rehabilitation and compensation, in accordance with law.
9. **Right Against Torture**: No person who is arrested or detained shall be subjected to physical or mental torture or to cruel, inhuman or degrading treatment.
10. **Right Against Preventive Detention**: No person shall be held under preventive detention unless there is a sufficient ground of the existence of an immediate threat to the sovereignty, territorial integrity, or public peace and order of Nepal.
11. **Right Against Untouchability and Discrimination**: No person shall be subjected to any form of untouchability or discrimination in any private and public places on grounds of their origin, caste, tribe, community, profession, occupation, or physical condition.

These fundamental rights are guaranteed to Nepali citizens, ensuring their dignity, freedom, and well-being.

In [45]:
%%time
response = qa.invoke("What are the powers and responsibilities of federal, provincial, and local governments?")
Markdown(response["result"])



Running pairwise ranking..
CPU times: user 1.81 s, sys: 113 ms, total: 1.92 s
Wall time: 4.26 s


**Powers and Responsibilities of Federal, Provincial, and Local Governments**

**Concurrent Powers of Federation, State, and Local Level:**

1. Cooperatives
2. Education, health, and newspapers
3. Health
4. Agriculture
5. Services such as electricity, water supply, irrigation
6. Service fee, charge, penalty, and royalty from natural resources, tourism fee
7. Forests, wildlife, birds, water uses, environment, ecology, and bio-diversity
8. Mines and minerals
9. Disaster management
10. Social security and poverty alleviation
11. Personal events, births, deaths, marriages, and statistics
12. Archaeology, ancient monuments, and museums
13. Landless squatters management
14. Royalty from natural resources
15. Motor vehicle permits

**Local Level Powers:**

1. Town police
2. Cooperative institutions
3. Operation of F.M.
4. Local taxes, service charge, fee, tourism fee, advertisement tax, business tax, land tax, penalty, entertainment tax, land revenue collection
5. Management of Local services
6. Collection of local statistics and records
7. Local level development plans and projects
8. Basic and secondary education
9. Basic health and sanitation
10. Local market management, environment protection, and bio-diversity
11. Local roads, rural roads, agro-roads, irrigation
12. Management of Village Assembly, Municipal Assembly, District Assembly, local courts, mediation, and arbitration
13. Local Government Services
14. Local records management
15. Distribution of house and land ownership certificates
16. Agriculture and animal husbandry, agro-products management, animal health, cooperatives
17. Management of senior citizens, persons with disabilities, and persons incapacitated
18. Collection of statistics of persons unemployed
19. Management, operation, and control of agricultural extension
20. Water supply, small hydropower projects, alternative energy
21. Disaster management
22. Protection of watersheds, wildlife, mines, and minerals
23. Protection and development of languages, cultures, and fine arts

**Additional Information:**

* The executive power of the Local Level is vested in the Village Executive or the Municipal Executive.
* The Village Executive and the Municipal Executive are responsible for issuing general directives, controlling, and regulating the governance of the Village Body and the Municipality.
* The Chairperson and Vice-Chairperson of the Village Executive are elected by the voters residing within the concerned Village Body area by secret ballots on the basis of one person one vote, in accordance with the first past the post electoral system.

In [46]:
%%time
response = qa.invoke("Who is Binit")
Markdown(response["result"])


Running pairwise ranking..
CPU times: user 2.27 s, sys: 105 ms, total: 2.37 s
Wall time: 2.87 s


**Answer:** I don't know who Binit is.

**Additional information:** The provided text does not mention a person named Binit. It appears to be a section of a constitution or legal document related to citizenship, government, and state affairs in Nepal.